## Initialize an iSNOBAL- and VW-ready dataset

We'll use vw-py's utilities to initialize a new isnobal- and vw-ready dataset. In order to build our dataset we need some information: 
    
* Start/end date
* The extreme northing and easting values (samps and bands in IPW speak)
* The resolution in northing and easting directions (dsamp/dline)
* The number of cells in both northing and easting driections

The example below is adapted from the [vw-py unit tests](https://github.com/VirtualWatershed/vw-py/blob/master/vwpy/test/test_netcdf.py#L427). We'll use the function [ncgen_from_template](https://github.com/VirtualWatershed/vw-py/blob/master/vwpy/netcdf.py) which can be found in the `vwpy.netcdf` module.

Briefly, this works by first building a CDL file from a [Jinja2](jinja.pocoo.org) template. See [CDL Syntax](http://www.unidata.ucar.edu/software/netcdf/netcdf/CDL-Syntax.html) and the [ncgen man page](http://www.unidata.ucar.edu/software/netcdf/netcdf/ncgen.html) for more CDL info. Then `ncgen_from_template` calls the command line `ncgen` function and loads the newly-generated dataset into Python.

In [95]:
# first, define our isnobal spatiotemporal parameters
isnobal_params = dict(
    # generate a 100x10x(n_timesteps) grid for each variable
    nlines=100, nsamps=10,
    # with a resolution of 1.0m each; samp is north-south, so it's negative
    dline=1.0, dsamp=-1.0,
    # set base fake origin (easting, northing) = (442, 88)
    bline=442, bsamp=88,
    # enter start time and timestep; janky, but need to use '01' and '00' 
    # to get proper ISO 8601 formatting if < 10
    year=2008, month=10, day='22', hour='05', dt='hours'
)

In [96]:
# now generate our netcdf
import sys, os
sys.path.append('../../')
if os.path.exists('test.nc'):
    os.remove('test.nc')

from vwpy.netcdf import ncgen_from_template

# don't need path to the template, that's already set to be 'vwpy/cdl'
nc = ncgen_from_template('ipw_in_template.cdl', 
                         'test.nc', **isnobal_params)

print isnobal_params

/tmp/cbdfe95b-53be-4481-b5ab-6c8dca2c52cd.cdl
{'bline': 442, 'hour': '05', 'nsamps': 10, 'month': 10, 'bsamp': 88, 'dsamp': -1.0, 'year': 2008, 'dt': 'hours', 'nlines': 100, 'day': '22', 'dline': 1.0}


#### Next: populate the netCDF with data

We'll insert data into just a few variables for an example. We can see what variables are now available either by inspecting the [CDL template](https://github.com/VirtualWatershed/vw-py/blob/master/vwpy/cdl/ipw_in_template.cdl) we just used, or we can inspect the Dataset's `variable` attribute, as shown below. 

One of the powerful features of netCDF is to transparently store data of any dimension. Some of our variables (time, easting, northing) are 1D. Others, including `z`, or altitude, are 2D variables; that is, they are spatially dependent, but time-independent. Finally, the climate variables that vary every hour and are spatially distributed are 3D arrays. 

In [97]:
# first, let's inspect what variables are available to us
nc.variables

OrderedDict([(u'time', <netCDF4.Variable at 0x10e9fb3b0>),
             (u'easting', <netCDF4.Variable at 0x10e9fb4d0>),
             (u'northing', <netCDF4.Variable at 0x10e9fb560>),
             (u'lat', <netCDF4.Variable at 0x10e9fb5f0>),
             (u'lon', <netCDF4.Variable at 0x10e9fb680>),
             (u'alt', <netCDF4.Variable at 0x10e9fb710>),
             (u'mask', <netCDF4.Variable at 0x10e9fb7a0>),
             (u'z', <netCDF4.Variable at 0x10e9fb830>),
             (u'z_0', <netCDF4.Variable at 0x10e9fb8c0>),
             (u'z_s', <netCDF4.Variable at 0x10e9fb950>),
             (u'rho', <netCDF4.Variable at 0x10e9fb9e0>),
             (u'T_s_0', <netCDF4.Variable at 0x10e9fba70>),
             (u'T_s', <netCDF4.Variable at 0x10e9fbb00>),
             (u'h2o_sat', <netCDF4.Variable at 0x10e9fbb90>),
             (u'm_pp', <netCDF4.Variable at 0x10e9fbc20>),
             (u'percent_snow', <netCDF4.Variable at 0x10e9fbcb0>),
             (u'rho_snow', <netCDF4.Variable at

In [98]:
# we can get metadata on any variable by inspecting the ncattrs
time = nc.variables['time']
time.ncattrs()

[u'long_name', u'standard_name']

In [99]:
# this is where the date went
time.getncattr('standard_name')

u'hours since 2008-10-22T05:00:00'

The standard name above refers to the [CF Conventions](http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html) standard name. By using this, other netCDF software tools can interpret the time variable, which unfortunately can only be represented as an integer index.

In [100]:
nc.ncattrs()

[u'data_tstep',
 u'output_frequency',
 u'bline',
 u'bsamp',
 u'dline',
 u'dsamp',
 u'max_z_s_0',
 u'z_u',
 u'z_T',
 u'z_g',
 u'documentation_link',
 u'description',
 u'generated_at',
 u'bands_name',
 u'bands_desc',
 u'exclude_layers']

In [101]:
nc.getncattr('bands_desc')

u'band index represents hours since 2008-10-22T05:00:00'

In [104]:
# let's create a fake DEM with some random data
import numpy as np
dem = abs(np.random.rand(isnobal_params['nlines'], isnobal_params['nsamps']))

import matplotlib.pyplot as plt
plt.imshow(dem)
plt.show()